### Pallet loading

Given a bunch of rectangles of the same size $(w,h)$, and a rectangular area of size $m\times n$, fit as many rectangles as possible into the are, with no overlap between rectangles. Model the presence of the lower left corner of the rectangle in the set of binary variables $q_{ij}$, and the occupance of grid points by the set of variables $Q_{ij}$

$$ 
H = -\sum_{i,j}^{m,n}q_{ij} + A \sum_{i,j}q_{ij}\sum_{l=i}^{i+w}\sum_{k=j}^{j+h}(1-Q_{ij}) + B\sum_{i,j}^{M,N}Q_{ij}+C\left(\sum_{i,j}^{m,n}Q_{ij}-s\sum_{i,j}^{m,n}q_{ij}\right)^2
$$

Here $s=w*h$, and $M = m + max(w, h), \text{ } N = n + max(w, h)$

In [23]:
import dimod
import neal
import numpy as np
from dwave.system import LeapHybridSampler
from itertools import product

m, n = 5, 5
w, h = 2, 2
area = w * h
M, N = m + w, n + w
memory_size = M * N + 2 * m * n
big_grid_size = M * N
small_grid_size = m * n
memory = np.arange(memory_size)

A = 1
B = 0.2
C = 0
num_fix = 1

Q = memory[:big_grid_size].reshape(M, N)
q = memory[big_grid_size: big_grid_size + small_grid_size].reshape(m, n)
t = memory[big_grid_size + small_grid_size:].reshape(m, n)
I = np.zeros((memory_size, memory_size, memory_size))
J = np.zeros((memory_size, memory_size))
H = np.zeros((memory_size,))

def set_up_linear_members():
    for i, j in product(range(M), range(N)):
        H[Q[i, j]] = B
    for i, j in product(range(m), range(n)):
        H[Q[i, j]] += C
        if num_fix is not None:
            H[q[i, j]] = C * area **2 + A * area - (2 * num_fix - 1)
        else:
            H[q[i, j]] = C * area **2 + A * area - 1
            
def set_up_quadratic_members():
    for i1, j1 in product(range(m), range(n)):
        for i2, j2 in product(range(i1, i1+w), range(j1, j1+h)):
            J[q[i1, j1], Q[i2, j2]] = -A
    for i1, j1 in product(range(m), range(n)):
        for i2, j2 in product(range(m), range(n)):
            J[q[i1, j1], Q[i2, j2]] -= 2 * C * area
            
    for i1, j1 in product(range(m), range(n)):
        for i2, j2 in product(range(m), range(n)):
            if (i1 == i2) and (j1 == j2):
                pass
            else:
                J[Q[i1, j1], Q[i2, j2]] = C 
                if num_fix is not None:
                    J[q[i1, j1], q[i2, j2]] = C * area **2 + 1
                else: 
                    J[q[i1, j1], q[i2, j2]] = C * area **2
                
def set_up_cubic_memebers():
    for i1, j1 in product(range(m), range(n)):
        for i2, j2 in product(range(m), range(n)):
            if (i2 >= i1) and (i2 < i1 + h) and (j2 >= j1 + h) and (j2 < j1 + w):
                I[q[i1, j1], t[i1, j1], Q[i2, j2]] = -1
            if (i2 >= i1 + h) and (i2 < i1 + w) and (j2 >= j1) and (j2 < j1 + h):
                I[q[i1, j1], t[i1, j1], Q[i2, j2]] = 1
                    
def get_dict_from_matrix(matrix):
    d = {}
    for i in product(*[range(s) for s in matrix.shape]):
        if matrix[i]:
            d[i] = matrix[i]
    return d

set_up_linear_members()
set_up_quadratic_members()
set_up_cubic_memebers()

poly = {}
poly.update(get_dict_from_matrix(I))
poly.update(get_dict_from_matrix(J))
poly.update(get_dict_from_matrix(H))

strength = 5

# bqm = dimod.make_quadratic(poly, strength, dimod.BINARY)
bqm = dimod.BinaryQuadraticModel(H, J, 0.0, dimod.BINARY)
sampler = neal.SimulatedAnnealingSampler()
response = sampler.sample(bqm, num_reads=1000)

In [29]:
result = np.array(list(response.lowest().first.sample.values()))

In [30]:
np.reshape(result[:49], (7, 7))

array([[0, 0, 0, 0, 0, 0, 0],
       [0, 0, 1, 1, 0, 0, 0],
       [0, 0, 1, 1, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0]], dtype=int8)

In [31]:
np.reshape(result[49:74], (5, 5))

array([[0, 0, 0, 0, 0],
       [0, 0, 1, 0, 0],
       [0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0]], dtype=int8)